## LlamaStack Vision API
This notebook demonstrates how to use the inference adaptor to interact with the SambaNova cloud-based vision models.

Please refer to the [llama-stack quickstart documentation](https://llama-stack.readthedocs.io/en/latest/getting_started/index.html) for further details.

## Setup

In [1]:
# Imports
import asyncio
import base64
import mimetypes
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.inference.event_logger import EventLogger
from llama_stack_client.types import UserMessage
from termcolor import cprint
import os

In [2]:
# Select model
model = 'sambanova/Llama-4-Maverick-17B-128E-Instruct'
# Select image path
image_path='../../images/SambaNova-dark-logo-1.png'

In [3]:
# Initialize client
LLAMA_STACK_PORT=8321
client = LlamaStackClient(
    base_url=f"http://localhost:{LLAMA_STACK_PORT}",
)

## Helper functions
Some utility functions to handle image processing and API interaction.

In [4]:
def encode_image_to_data_url(file_path: str) -> str:
    """
    Encode an image file to a data URL.

    Args:
        file_path: Path to the image file

    Returns:
        Data URL string
    """
    mime_type, _ = mimetypes.guess_type(file_path)
    if mime_type is None:
        raise ValueError("Could not determine MIME type of the file")

    with open(file_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode("utf-8")

    return f"data:{mime_type};base64,{encoded_string}"


## Chat with the image
The code below uses the Llama Stack Chat API to interact with the LLM.


In [5]:
def process_image_chat(client: LlamaStackClient, image_path: str, stream: bool = True):
    """
    Process an image through the LlamaStack Chat API.

    Args:
        client: Initialized client.
        image_path: Path to image file.capitalize.
        stream: Whether to stream the response.
    """
    data_url = encode_image_to_data_url(image_path)

    messages=[
        {
            'role': 'user', 
            'content': [
                {'type': 'image', 'image': {'url': {'uri': data_url}}},
                {'type': 'text', 'text': 'What does this image represent?'}
            ]
        },
    ]
    
    cprint("User> Sending image for analysis...", "green")
    response = client.inference.chat_completion(
        messages=messages,
        model_id=model,
        stream=stream,
    )

    if stream:
        text = ''
        for chunk in response:
            if chunk.event is not None:
                print(f'{chunk.event.delta.text}', end='', flush=True)
                text += chunk.event.delta.text
    else:
        print(
            f'Type: {type(response.completion_message.content)}, '
            f'Value:{response.completion_message.content}'
        )

In [6]:
# Chat with the iamge
process_image_chat(client=client, image_path=image_path, stream=True)

User> Sending image for analysis...
The image represents the logo of SambaNova Systems, a company that specializes in artificial intelligence (AI) and machine learning (ML) technology. The logo is a stylized representation of the company's name and brand identity.

**Key Elements:**

* **Logo:** The logo features a stylized letter "S" made up of curved lines, which are likely intended to represent the flow of data or information.
* **Color Scheme:** The logo is presented in two colors: orange and dark blue. The orange color is used for the stylized "S," while the dark blue is used for the text "SambaNova SYSTEMS."
* **Text:** The text "SambaNova SYSTEMS" is written in a clean, modern font next to the logo. The word "SambaNova" is prominently displayed in large letters, with "SYSTEMS" written in smaller letters underneath.
* **Trademark Symbol:** A registered trademark symbol is visible after the word "Nova," indicating that the name "SambaNova" is a registered trademark.

**Overall Imp